In [1]:
from google.colab import files

files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d shanks0465/braille-character-dataset


Saving kaggle.json to kaggle.json
  0% 0.00/1.30M [00:00<?, ?B/s]
100% 1.30M/1.30M [00:00<00:00, 114MB/s]


In [2]:
!unzip "/content/braille-character-dataset.zip"

Archive:  /content/braille-character-dataset.zip
  inflating: Braille Dataset/Braille Dataset/a1.JPG0dim.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG0rot.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG0whs.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG10dim.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG10rot.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG10whs.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG11dim.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG11rot.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG11whs.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG12dim.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG12rot.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG12whs.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG13dim.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG13rot.jpg  
  inflating: Braille Dataset/Braille Dataset/a1.JPG13whs.jpg  
  inflati

In [3]:
import os
from shutil import copyfile

os.mkdir('./dataset/')
alpha = 'a'
for i in range(0, 26): 
    os.mkdir('./dataset/' + alpha)
    alpha = chr(ord(alpha) + 1)

rootdir = '/content/Braille Dataset/Braille Dataset'
for file in os.listdir(rootdir):
    image_name = os.path.splitext(file)[0]
    letter = image_name[0]
    
    folder_path = './dataset/' + letter + '/' + image_name
    os.makedirs(folder_path, exist_ok=True) 
    
    image_path = os.path.join(rootdir, file)
    new_image_path = os.path.join(folder_path, file)
    
    copyfile(image_path, new_image_path)# Move the image to the new folder

In [10]:
#data Augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen= ImageDataGenerator(rescale=1./255,shear_range=10,validation_split=0.2)

train_generator = datagen.flow_from_directory('/content/dataset',
                                              target_size=(28,28),
                                              subset='training')

val_generator = datagen.flow_from_directory('/content/dataset',
                                            target_size=(28,28),
                                            subset='validation')

Found 1248 images belonging to 26 classes.
Found 312 images belonging to 26 classes.


In [11]:
import tensorflow
from keras.models import Sequential 
from keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

classifier=Sequential()

classifier.add(Conv2D(64,(3,3),input_shape=(28,28,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64,(3,3),activation='relu'))
classifier.add(Conv2D(64,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(64,activation='relu'))
classifier.add(Dense(128,activation='relu'))
classifier.add(Dense(256,activation='relu'))
classifier.add(Dense(26,activation='softmax'))

classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [12]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1024)             

In [13]:
model=classifier.fit(train_generator,
                               epochs=5,
                               validation_data=val_generator,)

Epoch 1/5
39/39 [==============================] - 6s 117ms/step - loss: 3.2616 - accuracy: 0.0401 - val_loss: 3.2552 - val_accuracy: 0.0609
Epoch 2/5
39/39 [==============================] - 5s 121ms/step - loss: 3.0212 - accuracy: 0.0962 - val_loss: 2.3974 - val_accuracy: 0.2179
Epoch 3/5
39/39 [==============================] - 5s 129ms/step - loss: 2.0487 - accuracy: 0.3958 - val_loss: 1.7033 - val_accuracy: 0.4679
Epoch 4/5
39/39 [==============================] - 4s 110ms/step - loss: 1.3955 - accuracy: 0.5737 - val_loss: 1.2140 - val_accuracy: 0.6603
Epoch 5/5
39/39 [==============================] - 5s 120ms/step - loss: 1.0741 - accuracy: 0.6603 - val_loss: 1.0614 - val_accuracy: 0.6891


In [16]:
classifier.save('model.h5')

In [15]:
#making the prediction
import numpy as np
from keras.preprocessing import image
from keras.utils import load_img, img_to_array
test_image=load_img("/content/Screenshot 2023-05-24 121025.png",target_size=(28,28))
test_image=img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
a=np.argmax(classifier.predict(test_image), axis=1)
a

1/1 [==============================] - 0s 27ms/step


array([21])